# Secretária de Segurança Pública - Ocorrências Criminais por Bairro

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Carregar os três arquivos CSV
file_path_2022 = '../data/SJCDadosCriminais_2022.csv'
file_path_2023 = '../data/SJCDadosCriminais_2023.csv'
file_path_2024 = '../data/SJCDadosCriminais_2024.csv'

data_2022 = pd.read_csv(file_path_2022)
data_2023 = pd.read_csv(file_path_2023)
data_2024 = pd.read_csv(file_path_2024)

# Concatenar os três DataFrames
data = pd.concat([data_2022, data_2023, data_2024], ignore_index=True)

# Obter todos os tipos de crimes únicos na coluna NATUREZA_APURADA
natureza_apurada_unique = sorted(data['NATUREZA_APURADA'].unique())

# Obter todos os bairros únicos na coluna BAIRRO e adicionar a opção "TODOS"
bairros_unique = ['TODOS'] + sorted(data['BAIRRO'].dropna().unique())

# Criar dropdowns para selecionar o tipo de crime e o bairro
crime_type_dropdown = widgets.Dropdown(
    options=natureza_apurada_unique,
    description='Tipo de Crime:',
    disabled=False,
)

bairro_dropdown = widgets.Dropdown(
    options=bairros_unique,
    description='Bairro:',
    disabled=False,
    value='TODOS'
)

# Função para gerar as estatísticas por bairro e ano com base no tipo de crime e bairro selecionado
def generate_crime_statistics(crime_type, bairro):
    clear_output(wait=True)  # Limpa a saída anterior
    display(crime_type_dropdown, bairro_dropdown, generate_button)  # Redesenha os widgets
    
    # Filtrar dados com base no tipo de crime selecionado
    data_filtered = data[data['NATUREZA_APURADA'] == crime_type]
    
    # Se um bairro específico for selecionado, filtrar por bairro
    if bairro != 'TODOS':
        data_filtered = data_filtered[data_filtered['BAIRRO'] == bairro]
    
    # Contagem de incidências por bairro e ano, ordenado por maior quantidade
    bairros_contagem = data_filtered.pivot_table(
        index='BAIRRO',
        columns='ANO_ESTATISTICA',
        aggfunc='size',
        fill_value=0
    ).reset_index()
    
    # Adicionar uma coluna de total geral por bairro, excluindo a coluna "BAIRRO"
    bairros_contagem['Total Geral'] = bairros_contagem.iloc[:, 1:].sum(axis=1)
    
    # Ordenar pelos bairros com mais ocorrências no total geral
    bairros_contagem = bairros_contagem.sort_values(by='Total Geral', ascending=False)
    
    # Exibir os bairros e suas contagens de ocorrências por ano e total geral
    estatisticas_html = f"""
    <p><b>Estatísticas para {crime_type}</b><br>
    Total de Eventos: {bairros_contagem['Total Geral'].sum()}</p>
    """
    estatisticas_html += bairros_contagem.to_html(index=False, justify='left')
    
    display(HTML(estatisticas_html))

# Criar um botão para gerar as estatísticas
generate_button = widgets.Button(description="Gerar Estatísticas")

# Conectar o botão à função
generate_button.on_click(lambda x: generate_crime_statistics(crime_type_dropdown.value, bairro_dropdown.value))

# Exibir widgets
display(crime_type_dropdown)
display(bairro_dropdown)
display(generate_button)


Dropdown(description='Tipo de Crime:', index=6, options=('APREENSÃO DE ENTORPECENTES', 'ESTUPRO', 'ESTUPRO DE …

Dropdown(description='Bairro:', options=('TODOS', '.', 'AGUA DE CANINDU', 'AGUA SOCA', 'AGUAS CANINDU II', 'AG…

Button(description='Gerar Estatísticas', style=ButtonStyle())

ANO_ESTATISTICA,BAIRRO,2022,2023,2024,Total Geral
,JARDIM SATÉLITE,79,71,16,166
,PARQUE INDUSTRIAL,45,41,8,94
,BOSQUE DOS EUCALIPTOS,32,43,13,88
,JARDIM MORUMBI,32,37,11,80
,JARDIM DAS INDÚSTRIAS,44,23,11,78
,JARDIM AMÉRICA,26,28,6,60
,CHÁCARAS REUNIDAS,21,25,8,54
,CENTRO,19,27,8,54
,VILA INDUSTRIAL,23,19,4,46
,CAMPO DOS ALEMÃES,11,24,10,45
